In [2]:
import numpy as np
import pandas as pd

In [3]:
def gaussian_kernel(X, A, opt={'sigma':0.1}):
    m = X.shape[0]
    sigma = opt['sigma']
    kernel_val = np.zeros((m, 1))
    dif = X - A
    Ki = np.sum(dif ** 2,axis=1) 
    kernel_val = np.exp(Ki /(-2 * sigma**2))
    return kernel_val

In [12]:
class SVM(object):
    def __init__(self, C=1, tol=0.001, kernel=gaussian_kernel,kernel_opt=None):
        self.C = C 
        self.tol = tol
        self.kernel = kernel
        self.kernel_opt = kernel_opt
        self.alphas = None
        self.X = None
        self.y = None
        self.kernel_val = None
        self.E = None

    def fit(self, X, y, epoch=40):
        m, n = X.shape
        self.b = 0
        self.alphas = abs(np.random.randn(m,1))
        self.X = X
        self.y = y
        self.kernel_val = np.empty((m, m))
        for i in range(m):
            self.kernel_val[:,i] = self.kernel(X, X[i,:],self.kernel_opt)
        self.E = [self.getE(i) for i in range(m)]

        flag = True
        alpha_changed = 0
        for i in range(epoch):
            if not (alpha_changed > 0 or flag): break
            alpha_changed = 0
            if flag:
                for j in range(m):
                    alpha_changed += self.update_alpha(j)
                i+=1
            else:
                support_alphas = [k for k, alpha in enumerate(self.alphas) if 0 < alpha < self.C]
                for j in support_alphas:
                    alpha_changed += self.update_alpha(j) 
                i+=1
            if flag:
                flag = False
            elif alpha_changed == 0:
                flag = True
    
    def predict(self, X):
        alpha_nonzero = np.nonzero(self.alphas)[0]
        support_alphas = self.alphas[alpha_nonzero]
        support_X = self.X[alpha_nonzero]
        support_y = self.y[alpha_nonzero]

        m = X.shape[0]
        num_support = support_X.shape[0]
        pred = np.empty((m,1))
        for i in range(m):
            kernel_val = self.kernel(support_X, X[i,:],self.kernel_opt)
            pred[i] = np.dot(kernel_val, support_alphas * support_y.reshape(num_support,1)) + self.b
        pred[pred>0] = 1
        pred[pred<=0] = 0
        return pred

    
    def update_alpha(self,i):
        """
        判断i样本是否满足KKT条件，满足则返回0，否则更新alpha，更新了返回1
        """
        Ei = self.getE(i)
        self.E[i] = Ei
        pred = self.y[i] * Ei
        if ((pred < -self.tol) and (self.alphas[i] < self.C)) or ((pred > self.tol) and (self.alphas[i] > 0)): #不满足KKT条件
            j,Ej = self.choose_second_alpha(i,Ei)
            alpha_i_old = self.alphas[i].copy()
            alpha_j_old = self.alphas[j].copy()
            #计算边界l和h
            if self.y[i] != self.y[j]:
                L = max(0,self.alphas[j]-self.alphas[i])
                H = min(self.C, self.C+self.alphas[j]-self.alphas[i])
            else:
                L = max(0,self.alphas[j]+self.alphas[i]-self.C)
                H = min(self.C,self.alphas[j]+self.alphas[i])
            if L == H:
                return 0

            eta = 2.0 * self.kernel_val[i,j] - self.kernel_val[i,i] - self.kernel_val[j,j]
            if eta >= 0:
                return 0

            alpha_j_new = alpha_j_old + self.y[j]*(Ei-Ej)/eta
            alpha_j_new = self.clip_alpha(alpha_j_new,L,H)
            if abs(alpha_j_new - alpha_j_old) < 0.00001:
                return 0

            alpha_i_new = alpha_i_old + self.y[j]*self.y[i]*(alpha_j_old - alpha_j_new)
            
            self.updateEk(i)
            self.updateEk(j)

            b1 = self.b - Ei - self.y[i]*self.kernel_val[i,i]*(alpha_i_old-alpha_i_new) + self.y[j]*self.kernel_val[i,j]*(alpha_j_old-alpha_j_new)
            b2 = self.b - Ej - self.y[i]*self.kernel_val[i,j]*(alpha_i_old-alpha_i_new) + self.y[j]*self.kernel_val[j,j]*(alpha_j_old-alpha_j_new)
            if 0 < alpha_i_new < self.C:
                self.b = b1
            elif 0 < alpha_j_new < self.C:
                self.b = b2
            else:
                self.b = (b1 + b2)/2
            self.alphas[i] = alpha_i_new
            self.alphas[j] = alpha_j_new
            return 1
        else:
            return 0

    def choose_second_alpha(self, i, Ei):
        j, Ej, max_step = -1, 0, 0
        valid_E = np.nonzero(self.E)[0]
        if len(valid_E) > 1:
            for k in valid_E:
                if k == i:
                    continue
                Ek = self.getE(k)
                delta_E = abs(Ei - Ek)
                if delta_E > max_step:
                    j, Ej, max_step = k, Ek, delta_E
        if j == -1:
            j = i
            while(j == i):
                np.random.seed()
                j = int(random.uniform(0,valid_E.shape[0]))
            Ej = self.getE(j)
        return j, Ej

    @staticmethod
    def clip_alpha(alpha, L, H):
        if alpha > H:
            return H
        if alpha < L:
            return L 
        return alpha

    def getE(self, i):
        dot = float(np.dot(self.kernel_val[:, i].T, self.alphas*(self.y.reshape(self.y.shape[0],1))))
        return dot + self.b - self.y[i]
        
    def updateEk(self, i):
        self.E[i] = self.getE(i)

    def get_kernek_val(self, X):
        m = X.shape[0]
        kernel_val = np.zeros((m,m))
        for i in range(m):
            kernel_val[:,i] = self.kernel(X, X[i,:])


In [13]:
def normalize_feature_z_score(df):
    df1 = df.iloc[:,:-1].apply(lambda column:(column - column.mean())/column.std())
    return df1.join(df.iloc[:,-1])

def normalize_feature(df):
    df1 = df.iloc[:,:-1].apply(lambda column:((column - column.min()) /( column.max() - column.min()) - 0.5))
    return df1.join(df.iloc[:,-1])          

In [14]:
data = pd.read_csv("pima_indian.csv")


In [15]:
df_nol = normalize_feature(data)
X = df_nol.iloc[:450,:-1]
y = df_nol.iloc[:450,-1]
X_val = df_nol.iloc[450:600,:-1]
y_val = df_nol.iloc[450:600,-1]
X_test = df_nol.iloc[601:,:-1]
y_test = df_nol.iloc[601:, -1]


In [33]:
c = 0.7
sigma = 1.5
model = SVM(C=c,kernel_opt={'sigma':sigma})
model.fit(np.array(X), np.array(y),epoch=50)
pred = model.predict(np.array(X_val))
acc = np.mean(y_val.values == pred)
print(acc)

0.7496
